In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
import tqdm
import shutil

In [3]:
#clone the url https://github.com/ieee8023/covid-chestxray-dataset
base_dir = 'D:/covid-chestxray-dataset' #give the clone github url path
os.listdir(base_dir)

['.git',
 '.github',
 '.gitignore',
 'annotations',
 'covid_images',
 'docs',
 'images',
 'metadata.csv',
 'README.md',
 'requirements.txt',
 'SCHEMA.md',
 'scripts',
 'tests',
 'volumes']

In [4]:
df = pd.read_csv(os.path.join(base_dir, 'metadata.csv'))

In [5]:
df.head()

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [6]:
df.columns

Index(['patientid', 'offset', 'sex', 'age', 'finding', 'RT_PCR_positive',
       'survival', 'intubated', 'intubation_present', 'went_icu', 'in_icu',
       'needed_supplemental_O2', 'extubated', 'temperature', 'pO2_saturation',
       'leukocyte_count', 'neutrophil_count', 'lymphocyte_count', 'view',
       'modality', 'date', 'location', 'folder', 'filename', 'doi', 'url',
       'license', 'clinical_notes', 'other_notes', 'Unnamed: 29'],
      dtype='object')

In [7]:
df['finding'].unique()

array(['Pneumonia/Viral/COVID-19', 'Pneumonia', 'Pneumonia/Viral/SARS',
       'Pneumonia/Fungal/Pneumocystis',
       'Pneumonia/Bacterial/Streptococcus', 'No Finding',
       'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli',
       'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella',
       'Unknown', 'Pneumonia/Lipoid', 'Pneumonia/Viral/Varicella',
       'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Mycoplasma',
       'Pneumonia/Viral/Influenza', 'todo', 'Tuberculosis',
       'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Fungal/Aspergillosis',
       'Pneumonia/Viral/Herpes ', 'Pneumonia/Aspiration',
       'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Viral/MERS-CoV',
       'Pneumonia/Bacterial/Staphylococcus/MRSA'], dtype=object)

In [8]:
#creating directory to store images if not already present
folder_name = os.path.join(base_dir, 'covid_images')
try:
    os.mkdir(folder_name)
except OSError as error:
    print(error)
    pass  

#if covid-19 is present in finding column of dataframe then we will store that image in our covid_images directory using url column
count = 0
for i in tqdm.tqdm(range(len(df))):
    if('COVID-19' in df['finding'][i]):
        im_name = df['filename'][i]
        if(im_name.split('.')[-1].lower() in ['jpg', 'jpeg', 'png']):   #only adding image if file extension is jpg, jpeg or png
            im = cv2.imread(os.path.join(base_dir, 'images', im_name))  #give your path to images folder
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)  #open cv loads image in BGR format but we expect in RGB so swapping channels
            im = cv2.resize(im, (250,250), interpolation = cv2.INTER_NEAREST)
            # print(im.shape)
            img_path = os.path.join(folder_name, str(count) + ".jpg")
            cv2.imwrite(img_path, im)
            count += 1

  0%|                                                                                          | 0/950 [00:00<?, ?it/s]

[WinError 183] Cannot create a file when that file already exists: 'D:/covid-chestxray-dataset\\covid_images'


100%|████████████████████████████████████████████████████████████████████████████████| 950/950 [00:47<00:00, 19.99it/s]


In [11]:
#moving images from one folder to another 
def move_images(input_folder, output_folder, r = (None, None)): #give input folder, output folder(where to store image) and start and end of images
    #provide r as a tupple (start_ind, end_ind)
    if r == (None, None):
        r = (1, len(os.listdir(input_folder)))  #taking all the images
        
    count = 1
    for im in tqdm.tqdm(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, im)):
            if(count >= r[0] and count <= r[1]):
                shutil.copy(os.path.join(input_folder, im), os.path.join(output_folder, im))
            count += 1
            
        if(count > r[1]):
            break

In [16]:
input_folder = 'D:/covid-chestxray-dataset/covid_images'  #give your folder paths
output_folder = 'D:/covid_session_dataset/covid19'
move_images(input_folder, output_folder)

 98%|████████████████████████████████████████████████████████████████████████████▌ | 553/563 [00:00<00:00, 1329.69it/s]


In [17]:
input_folder = r'D:\Downloads\kaggle_covid19\chest_xray\train\PNEUMONIA'
output_folder = r'D:\covid_session_dataset\pneumonia'
move_images(input_folder, output_folder, (1, 1500))

 39%|██████████████████████████████                                                | 1495/3875 [00:50<00:42, 56.45it/s]

In [20]:
input_folder = r'D:\Downloads\kaggle_covid19\chest_xray\train\NORMAL'
output_folder = r'D:\covid_session_dataset\normal'

if not os.path.isdir(output_folder):
    try:
        os.mkdir(output_folder)
    except OSError as err:
        print(err)
        pass
    
move_images(input_folder, output_folder) 


 92%|████████████████████████████████████████████████████████████████████████      | 1238/1341 [01:00<00:05, 18.37it/s]


100%|█████████████████████████████████████████████████████████████████████████████▊| 1338/1341 [01:17<00:00, 23.27it/s]

In [29]:
#resizing and storing in train folder
def resize_image(input_folder, output_folder, size):
    count = 0
    for im in os.listdir(input_folder):
        imag = cv2.imread(os.path.join(input_folder, im))
        imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
        imag = cv2.resize(imag, size, interpolation = cv2.INTER_NEAREST)
        im_path = os.path.join(output_folder, str(count) + ".jpg")
        #if want image name not to be changed comment above line and uncomment below one
        #im_path = os.path.join(output_folder, im)
        cv2.imwrite(im_path, imag)
        count += 1

In [30]:
input_folder = r'D:\covid_session_dataset\pneumonia'
output_folder = r'D:\covid_session_dataset\train\pneumonia'
resize_image(input_folder, output_folder, (250, 250))

In [31]:
input_folder = r'D:\covid_session_dataset\normal'
output_folder = r'D:\covid_session_dataset\train\normal'
resize_image(input_folder, output_folder, (250, 250))

In [9]:
#basic image augmentation
# positive values to increase brightness
#negative values to decrease brightness
def brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    h, s, v = cv2.split(hsv)
    if(value > 0):
        lim = 255 - value
        v[v > lim] = 255
        v[v <= lim] += value
    if(value < 0):
        v[v < abs(value)] = 0
        v[v >= abs(value)] -= abs(value)
        
    hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return img

In [12]:
def augment(input_folder, output_folder, suffix, value, size, r=(None, None)):
    if r == (None, None):
        r = (1, len(os.listdir(input_folder)))  #taking all the images
        
    count = 1
    for im in os.listdir(input_folder):
        if(count >= r[0] and count <= r[1]):
            img = cv2.imread(os.path.join(input_folder, im))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = brightness(img, value)
            img = cv2.resize(img, size, interpolation = cv2.INTER_NEAREST)
            img_path = os.path.join(output_folder, im.split('.')[0] + "_" + suffix + ".jpg")
            cv2.imwrite(img_path, img)
        count += 1
        if(count > r[1]):
            break

In [65]:
input_folder = r'D:\covid_session_dataset\covid19'
output_folder = r'D:\covid_session_dataset\train\covid19'
augment(input_folder, output_folder, "dark", -30, (250,250), (1, 500))

In [13]:
input_folder = r'D:\covid_session_dataset\normal'
output_folder = r'D:\covid_session_dataset\train\normal'
augment(input_folder, output_folder, "bright", 20, (250, 250) , (1, 159))